# CDC: Monkeypox cases, by state

In [1]:
import pandas as pd
import us
import urllib.request, json
import datetime as dt

In [2]:
today = pd.Timestamp.now(tz='America/Los_Angeles').strftime("%Y-%m-%d")
time = pd.Timestamp.now(tz='America/Los_Angeles').strftime("%-I:%M %p")

---

## State populations

#### Figures from ACS 2020, 5-year estimates

In [3]:
states_pop = pd.read_csv('data/raw/states_population.csv')

#### Map FIPS codes to each state

In [4]:
states_pop['state_fips'] = states_pop['state'].map(us.states.mapping('name', 'fips'))

---

## CDC Monkeypox

#### Aggregated by state

In [5]:
states_src = pd.read_csv('https://www.cdc.gov/wcms/vizdata/poxvirus/monkeypox/data/USmap_counts.csv')

In [6]:
states_src.columns = states_src.columns.str.lower()

In [7]:
states_src.head(1)

,location,cases,case range,asof
0,Alabama,90,51 to 100,Data as of September 15 2022 at 2:00 pm EDT


#### Map FIPS codes and AP abbrevations to each state

In [8]:
states_src['state_fips'] = states_src['location'].map(us.states.mapping('name', 'fips'))
states_src['state_ap'] = states_src['location'].map(us.states.mapping('name', 'ap_abbr'))
states_src['state_postal'] = states_src['location'].map(us.states.mapping('name', 'abbr'))

---

## Merge

#### Population and cases to create a rate

In [9]:
src = pd.merge(states_src, states_pop, on=['state_fips'])

In [10]:
df = src[(src['location'] != 'Total') | (src['location'] != 'Non-US Resident')].copy()

In [11]:
df['cases'] = df['cases'].astype(int)

In [12]:
df['cases_per_million'] = ((df['cases'] / df['pop_acs_2020_5tr'])*1000000).astype(float).round(2)

In [13]:
df.drop(['case range'], axis=1, inplace=True)

In [14]:
df['cases'].sum()

23114

---

## Exports

In [ ]:
# df.to_csv(f'data/processed/monkeypox_cases_states_cdc_{today}.csv', index=False)
# df.to_json(f'data/processed/monkeypox_cases_states_cdc_{today}.json', orient='records', indent=4)